# colors

> Semantic color system builders for daisyUI components

In [ ]:
#| default_exp builders.colors

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Dict, List, Union, Optional, Any
from enum import Enum
from cjm_fasthtml_tailwind.core.base import (
    BaseUtility, BaseFactory, ModifierMixin,
    TailwindValue, is_custom_property, is_arbitrary_value
)

## Semantic Color Enums

Define enums for the semantic color names used by daisyUI:

In [ ]:
#| export
class SemanticColorBrand(str, Enum):
    """daisyUI semantic brand color names."""
    # Brand colors
    PRIMARY = "primary"
    SECONDARY = "secondary"
    ACCENT = "accent"
    NEUTRAL = "neutral"

In [ ]:
#| export
class SemanticColorStatus(str, Enum):
    """daisyUI semantic status color names."""
    # Status colors
    INFO = "info"
    SUCCESS = "success"
    WARNING = "warning"
    ERROR = "error"

In [ ]:
#| export
class SemanticColorBase(str, Enum):
    """daisyUI semantic base color names."""
    # Base colors for surfaces and backgrounds
    BASE_100 = "base-100"
    BASE_200 = "base-200"
    BASE_300 = "base-300"
    BASE_CONTENT = "base-content"

In [ ]:
#| export
class SemanticColorContent(str, Enum):
    """daisyUI semantic content color names."""    
    # Content colors (text on colored backgrounds)
    PRIMARY_CONTENT = "primary-content"
    SECONDARY_CONTENT = "secondary-content"
    ACCENT_CONTENT = "accent-content"
    NEUTRAL_CONTENT = "neutral-content"
    INFO_CONTENT = "info-content"
    SUCCESS_CONTENT = "success-content"
    WARNING_CONTENT = "warning-content"
    ERROR_CONTENT = "error-content"

In [ ]:
#| export
class SemanticColor(str, Enum):
    """daisyUI semantic color names."""
    
    # Use _ignore_ to exclude loop variables
    _ignore_ = ['member', 'enum_cls']
    
    # Dynamically add members from semantic color enums
    for enum_cls in [SemanticColorBrand, SemanticColorBase, SemanticColorStatus, SemanticColorContent]:
        for member in enum_cls:
            locals()[member.name] = member.value

## Color Value Types

Define types for semantic color values:

In [ ]:
#| export
# Type for semantic color values including SemanticColor enum and string
SemanticColorValue = Union[SemanticColor, str]

## Color Validation

Functions to validate semantic color values:

In [ ]:
#| export
def is_valid_semantic_color(
    value: str  # The value to check
) -> bool:  # True if value is a valid daisyUI semantic color
    """Check if a value is a valid daisyUI semantic color."""
    return value in [c.value for c in SemanticColor]

## Colored Utility Class for daisyUI

A utility class that supports daisyUI semantic colors with opacity:

In [ ]:
#| export
class ColoredUtilityDaisyUI(BaseUtility, ModifierMixin):
    """Utility class with daisyUI semantic color and opacity support."""
    
    def __init__(
        self,
        prefix: str,  # The utility prefix (e.g., 'bg', 'text', 'border')
        color: Optional[SemanticColorValue] = None,  # The semantic color value
        opacity: Optional[Union[int, str]] = None  # Optional opacity value (0-100 or arbitrary)
    ):
        """Initialize with prefix, optional semantic color, and optional opacity."""
        super().__init__(prefix)
        self._color = color
        self._opacity = opacity
    
    def _format_value(
        self,
        value: TailwindValue  # The value to format (semantic color)
    ) -> str:  # The formatted color value
        """
        Format semantic color value according to Tailwind conventions.
        Handles semantic colors and arbitrary values.
        """
        value_str = str(value)
        
        # Handle SemanticColor enum
        if isinstance(value, SemanticColor):
            return value.value
        
        # Handle valid semantic colors
        if is_valid_semantic_color(value_str):
            return value_str
        
        # Handle custom properties
        if is_custom_property(value_str):
            return f"({value_str})"
        
        # Handle arbitrary values (including hex colors, rgb, etc.)
        if is_arbitrary_value(value_str) or not is_valid_semantic_color(value_str):
            return f"[{value_str}]"
        
        # Return as-is for other cases
        return value_str
    
    def _build_class(
        self,
        value: Optional[SemanticColorValue] = None  # Optional color value to use
    ) -> str:  # The complete CSS class string
        """Build the complete CSS class string with optional opacity."""
        # Use provided value or stored color
        color_value = value if value is not None else self._color
        
        if color_value is None:
            return self.prefix
        
        # Format the color part
        formatted_color = self._format_value(color_value)
        base_class = f"{self.prefix}-{formatted_color}"
        
        # Add opacity if specified
        if self._opacity is not None:
            base_class = f"{base_class}/{self._opacity}"
        
        # Apply modifiers
        if self._modifiers:
            return ":".join(self._modifiers + [base_class])
        
        return base_class
    
    def opacity(
        self,
        value: Union[int, str]  # Opacity value (0-100 or arbitrary)
    ) -> 'ColoredUtilityDaisyUI':  # A new instance with opacity set
        """Return a new ColoredUtilityDaisyUI instance with opacity value."""
        # Create a new instance to avoid modifying the original
        new_instance = ColoredUtilityDaisyUI(self.prefix, self._color, value)
        new_instance._modifiers = self._modifiers.copy() if self._modifiers else []
        new_instance._value = self._value
        return new_instance

## Colored Factory for daisyUI

Factory for creating semantic color-based utilities with convenient API:

In [ ]:
#| export
class ColoredFactoryDaisyUI(BaseFactory):
    """Factory for creating daisyUI semantic color-based utilities."""
    
    def __new__(cls, prefix: str, doc: Optional[str] = None):
        """Create a new instance with dynamic properties for tab-completion."""

        # If this is a subclass, just create a normal instance
        # Subclasses will add properties in __init__ if needed
        if cls is not ColoredFactoryDaisyUI:
            return object.__new__(cls)
        
        # Create a unique subclass for this instance
        class_name = f"ColoredFactoryDaisyUI_{prefix}_{id(prefix)}"
        
        # Create class attributes for each semantic color
        class_attrs = {}
        
        # Add properties for each semantic color
        for color in SemanticColor:
            # Convert enum name to property name (e.g., BASE_100 -> base_100)
            prop_name = color.name.lower()
            
            # Create a property that returns a ColoredUtilityDaisyUI instance
            def make_property(color_value):
                def getter(self):
                    return ColoredUtilityDaisyUI(self.prefix, color_value)
                return property(getter)
            
            class_attrs[prop_name] = make_property(color.value)
        
        # Create the dynamic subclass
        DynamicFactory = type(class_name, (ColoredFactoryDaisyUI,), class_attrs)
        
        # Create and return an instance of the dynamic class
        instance = object.__new__(DynamicFactory)
        return instance
    
    def __init__(
        self,
        prefix: str,  # The utility prefix (e.g., 'bg', 'text', 'border')
        doc: Optional[str] = None  # Optional documentation string
    ):
        """Initialize factory with prefix and documentation."""
        doc = doc or f"Factory for {prefix} semantic color utilities"
        super().__init__(doc)
        self.prefix = prefix

        # Add dynamic properties if we have values and this is a subclass
        # or if this is a direct instance that didn't get properties in __new__
        if self._values and (self.__class__.__name__ == 'ColoredFactoryDaisyUI' or 
                           not any(isinstance(getattr(self.__class__, attr, None), property) 
                                 for attr in dir(self.__class__))):
            self._add_dynamic_properties(self._values)


    def _add_dynamic_properties(self, values_dict: Dict[str, str]):
        """Add properties dynamically to the instance for tab-completion."""
        # Create a new class that inherits from the current class
        current_class = self.__class__
        class_name = f"{current_class.__name__}_Dynamic_{id(values_dict)}"
        
        # Create class attributes for each value
        class_attrs = {}
        
        
        # Copy existing class attributes
        for attr_name in dir(current_class):
            if not attr_name.startswith('_'):
                attr = getattr(current_class, attr_name)
                if not callable(attr) or isinstance(attr, property):
                    class_attrs[attr_name] = attr

        # Add properties for each semantic color
        for color in SemanticColor:
            # Convert enum name to property name (e.g., BASE_100 -> base_100)
            prop_name = color.name.lower()
            
            # Create a property that returns a ColoredUtilityDaisyUI instance
            def make_property(color_value):
                def getter(self):
                    return ColoredUtilityDaisyUI(self.prefix, color_value)
                return property(getter)
            
            class_attrs[prop_name] = make_property(color.value)
        
        # Create the dynamic subclass
        DynamicClass = type(class_name, (current_class,), class_attrs)
        
        # Change the instance's class to the dynamic one
        self.__class__ = DynamicClass
    
    def __call__(
        self,
        color: SemanticColorValue,  # The semantic color
        opacity: Optional[Union[int, str]] = None  # Optional opacity
    ) -> ColoredUtilityDaisyUI:  # A new ColoredUtilityDaisyUI instance
        """Create a ColoredUtilityDaisyUI with specified color and optional opacity."""
        return ColoredUtilityDaisyUI(self.prefix, color, opacity)
    
    def __getattr__(
        self,
        name: str  # Attribute name (for arbitrary values)
    ) -> ColoredUtilityDaisyUI:  # ColoredUtilityDaisyUI for arbitrary values
        """
        Handle attribute access for arbitrary color values.
        Note: For semantic colors, use the structured approach: bg_dui.primary
        """
        # Only handle arbitrary color values here
        return ColoredUtilityDaisyUI(self.prefix, name.replace("_", "-"))
    
    def get_info(
        self
    ) -> Dict[str, Any]:  # Dictionary with factory information
        """Get detailed information about this semantic color factory."""
        semantic_colors = [c.value for c in SemanticColor]
        
        return {
            'description': self._doc,
            'valid_inputs': {
                'semantic_colors': semantic_colors,
                'usage_examples': [
                    f'{self.prefix}_dui.primary',
                    f'{self.prefix}_dui.secondary',
                    f'{self.prefix}_dui.base_100',
                    f'{self.prefix}_dui.primary.opacity(50)',
                    f'{self.prefix}_dui("primary", opacity=75)',
                    f'{self.prefix}_dui("#ff0000")',
                    f'{self.prefix}_dui("--custom-color")'
                ]
            },
            'options': {
                'prefix': self.prefix,
                'supports_opacity': True,
                'opacity_syntax': 'color/opacity'
            }
        }

## Examples

Test the semantic color system with various use cases:

In [ ]:
# Test basic semantic color usage
bg_dui = ColoredFactoryDaisyUI("bg", "Background color utilities for daisyUI semantic colors")

# Test semantic colors
assert str(bg_dui.primary) == "bg-primary"
assert str(bg_dui.secondary) == "bg-secondary"
assert str(bg_dui.accent) == "bg-accent"
assert str(bg_dui.neutral) == "bg-neutral"

In [ ]:
# Test base colors
assert str(bg_dui.base_100) == "bg-base-100"
assert str(bg_dui.base_200) == "bg-base-200"
assert str(bg_dui.base_300) == "bg-base-300"
assert str(bg_dui.base_content) == "bg-base-content"

In [ ]:
# Test status colors
assert str(bg_dui.info) == "bg-info"
assert str(bg_dui.success) == "bg-success"
assert str(bg_dui.warning) == "bg-warning"
assert str(bg_dui.error) == "bg-error"

In [ ]:
# Test content colors
assert str(bg_dui.primary_content) == "bg-primary-content"
assert str(bg_dui.secondary_content) == "bg-secondary-content"
assert str(bg_dui.accent_content) == "bg-accent-content"
assert str(bg_dui.neutral_content) == "bg-neutral-content"

### Semantic Color Opacity

Control the opacity of semantic colors:

In [ ]:
# Test opacity modifiers
assert str(bg_dui.primary.opacity(50)) == "bg-primary/50"
assert str(bg_dui.secondary.opacity(75)) == "bg-secondary/75"
assert str(bg_dui.base_100.opacity(10)) == "bg-base-100/10"
assert str(bg_dui.base_content.opacity(70)) == "bg-base-content/70"

# Test arbitrary opacity values
assert str(bg_dui.success.opacity("[0.87]")) == "bg-success/[0.87]"
assert str(bg_dui.error.opacity("(--my-opacity)")) == "bg-error/(--my-opacity)"

### Arbitrary Color Values

Support for custom colors alongside semantic colors:

In [ ]:
# Test arbitrary color values
assert str(bg_dui("#ff0000")) == "bg-[#ff0000]"
assert str(bg_dui("rgb(255, 0, 0)")) == "bg-[rgb(255, 0, 0)]"
assert str(bg_dui("hsl(0, 100%, 50%)")) == "bg-[hsl(0, 100%, 50%)]"

# Test CSS custom properties
assert str(bg_dui("--custom-bg")) == "bg-(--custom-bg)"
assert str(bg_dui("--theme-primary")) == "bg-(--theme-primary)"

# Test with opacity
assert str(bg_dui("--custom-bg", opacity=50)) == "bg-(--custom-bg)/50"

### Factory Call Syntax

Test using the factory with call syntax:

In [ ]:
# Test factory call syntax
assert str(bg_dui("primary")) == "bg-primary"
assert str(bg_dui("secondary", opacity=75)) == "bg-secondary/75"
assert str(bg_dui(SemanticColor.ACCENT)) == "bg-accent"
assert str(bg_dui(SemanticColor.BASE_100, opacity=50)) == "bg-base-100/50"

## Test Functions

Comprehensive test functions following the project's naming convention:

In [ ]:
#| export
def test_colors_semantic_enum_examples():
    """Test semantic color enum."""
    # Test SemanticColor enum
    assert SemanticColor.PRIMARY.value == "primary"
    assert SemanticColor.SECONDARY.value == "secondary"
    assert SemanticColor.ACCENT.value == "accent"
    assert SemanticColor.NEUTRAL.value == "neutral"
    
    # Test base colors
    assert SemanticColor.BASE_100.value == "base-100"
    assert SemanticColor.BASE_200.value == "base-200"
    assert SemanticColor.BASE_300.value == "base-300"
    assert SemanticColor.BASE_CONTENT.value == "base-content"
    
    # Test status colors
    assert SemanticColor.INFO.value == "info"
    assert SemanticColor.SUCCESS.value == "success"
    assert SemanticColor.WARNING.value == "warning"
    assert SemanticColor.ERROR.value == "error"
    
    # Verify we have all semantic colors
    assert len(SemanticColor) == 20  # 4 brand + 4 base + 4 status + 8 content colors

# Run the test
test_colors_semantic_enum_examples()

In [ ]:
#| export
def test_colors_validation_examples():
    """Test semantic color validation functions."""
    # Test valid semantic colors
    assert is_valid_semantic_color("primary") == True
    assert is_valid_semantic_color("secondary") == True
    assert is_valid_semantic_color("base-100") == True
    assert is_valid_semantic_color("primary-content") == True
    assert is_valid_semantic_color("error") == True
    
    # Test invalid colors
    assert is_valid_semantic_color("red-500") == False
    assert is_valid_semantic_color("blue") == False
    assert is_valid_semantic_color("invalid") == False
    assert is_valid_semantic_color("#ff0000") == False

# Run the test
test_colors_validation_examples()

In [ ]:
#| export
def test_colors_factory_examples():
    """Test ColoredFactoryDaisyUI with various semantic color specifications."""
    bg_dui = ColoredFactoryDaisyUI("bg")
    
    # Test brand colors with structured approach
    assert str(bg_dui.primary) == "bg-primary"
    assert str(bg_dui.secondary) == "bg-secondary"
    assert str(bg_dui.accent) == "bg-accent"
    assert str(bg_dui.neutral) == "bg-neutral"
    
    # Test base colors
    assert str(bg_dui.base_100) == "bg-base-100"
    assert str(bg_dui.base_200) == "bg-base-200"
    assert str(bg_dui.base_300) == "bg-base-300"
    assert str(bg_dui.base_content) == "bg-base-content"
    
    # Test status colors
    assert str(bg_dui.info) == "bg-info"
    assert str(bg_dui.success) == "bg-success"
    assert str(bg_dui.warning) == "bg-warning"
    assert str(bg_dui.error) == "bg-error"
    
    # Test with call syntax
    assert str(bg_dui("primary")) == "bg-primary"
    assert str(bg_dui("base-100", opacity=50)) == "bg-base-100/50"

# Run the test
test_colors_factory_examples()

In [ ]:
#| export
def test_colors_opacity_examples():
    """Test opacity modifiers with semantic color utilities."""
    bg_dui = ColoredFactoryDaisyUI("bg")
    
    # Test opacity with semantic colors
    assert str(bg_dui.primary.opacity(50)) == "bg-primary/50"
    assert str(bg_dui.secondary.opacity(75)) == "bg-secondary/75"
    assert str(bg_dui.base_100.opacity(10)) == "bg-base-100/10"
    assert str(bg_dui.base_content.opacity(70)) == "bg-base-content/70"
    
    # Test opacity with arbitrary values
    assert str(bg_dui.primary.opacity("[0.87]")) == "bg-primary/[0.87]"
    assert str(bg_dui.error.opacity("(--my-opacity)")) == "bg-error/(--my-opacity)"
    
    # Test opacity in factory call
    assert str(bg_dui("success", opacity=25)) == "bg-success/25"
    
    # Test that opacity doesn't persist (bug fix)
    primary = bg_dui.primary
    assert str(primary) == "bg-primary"
    assert str(primary.opacity(50)) == "bg-primary/50"
    assert str(primary) == "bg-primary"  # Should still be without opacity

# Run the test
test_colors_opacity_examples()

In [ ]:
#| export
def test_colors_arbitrary_examples():
    """Test arbitrary color values and custom properties."""
    bg_dui = ColoredFactoryDaisyUI("bg")
    
    # Test hex colors
    assert str(bg_dui("#ff0000")) == "bg-[#ff0000]"
    assert str(bg_dui("#00ff00")) == "bg-[#00ff00]"
    assert str(bg_dui("#0000ff")) == "bg-[#0000ff]"
    
    # Test RGB/HSL functions
    assert str(bg_dui("rgb(255, 0, 0)")) == "bg-[rgb(255, 0, 0)]"
    assert str(bg_dui("hsl(0, 100%, 50%)")) == "bg-[hsl(0, 100%, 50%)]"
    assert str(bg_dui("rgba(255, 0, 0, 0.5)")) == "bg-[rgba(255, 0, 0, 0.5)]"
    
    # Test CSS custom properties
    assert str(bg_dui("--custom-bg")) == "bg-(--custom-bg)"
    assert str(bg_dui("--theme-primary")) == "bg-(--theme-primary)"

# Run the test
test_colors_arbitrary_examples()

## Practical Examples

Test with multiple utility prefixes to show the semantic color system is reusable:

In [ ]:
#| export
def test_colors_multiple_utilities_examples():
    """Test semantic color system with multiple utility types."""
    # Create factories for different utilities
    bg_dui = ColoredFactoryDaisyUI("bg", "Background color utilities for daisyUI semantic colors")
    text_dui = ColoredFactoryDaisyUI("text", "Text color utilities for daisyUI semantic colors")
    border_dui = ColoredFactoryDaisyUI("border", "Border color utilities for daisyUI semantic colors")
    
    # Test same semantic color across different utilities
    assert str(bg_dui.primary) == "bg-primary"
    assert str(text_dui.primary) == "text-primary"
    assert str(border_dui.primary) == "border-primary"
    
    # Test with opacity
    assert str(bg_dui.secondary.opacity(50)) == "bg-secondary/50"
    assert str(text_dui.secondary.opacity(50)) == "text-secondary/50"
    assert str(border_dui.secondary.opacity(50)) == "border-secondary/50"
    
    # Test base colors
    assert str(bg_dui.base_100) == "bg-base-100"
    assert str(text_dui.base_content) == "text-base-content"
    assert str(border_dui.base_300) == "border-base-300"

# Run the test
test_colors_multiple_utilities_examples()

In [ ]:
#| export
def test_colors_practical_usage_examples():
    """Test practical usage patterns with FastHTML components."""
    from fasthtml.common import Div, Span, P, Button
    from cjm_fasthtml_tailwind.core.base import combine_classes
    
    bg_dui = ColoredFactoryDaisyUI("bg")
    text_dui = ColoredFactoryDaisyUI("text")
    border_dui = ColoredFactoryDaisyUI("border")
    
    # Create a daisyUI themed card component
    card = Div(
        P("Hello daisyUI!", cls=str(text_dui.primary)),
        cls=combine_classes(
            bg_dui.base_100,
            border_dui.base_300,
            "rounded-lg p-4 border"
        )
    )
    
    # Verify classes
    assert "bg-base-100" in card.attrs['class']
    assert "border-base-300" in card.attrs['class']
    assert card.children[0].attrs['class'] == "text-primary"
    
    # Create a semi-transparent overlay with semantic colors
    overlay = Div(
        cls=str(bg_dui.neutral.opacity(50))
    )
    assert overlay.attrs['class'] == "bg-neutral/50"
    
    # Create a button with semantic colors
    button = Button(
        "Click me",
        cls=combine_classes(
            bg_dui.primary,
            text_dui.primary_content,
            "px-4 py-2 rounded"
        )
    )
    assert "bg-primary" in button.attrs['class']
    assert "text-primary-content" in button.attrs['class']

# Run the test
test_colors_practical_usage_examples()

In [ ]:
#| export
def test_colors_modifier_examples():
    """Test semantic color utilities with modifiers for conditional styling."""
    bg_dui = ColoredFactoryDaisyUI("bg")
    text_dui = ColoredFactoryDaisyUI("text")
    border_dui = ColoredFactoryDaisyUI("border")
    
    # Test hover states
    assert str(bg_dui.primary.hover) == "hover:bg-primary"
    assert str(text_dui.secondary.hover) == "hover:text-secondary"
    assert str(border_dui.accent.hover) == "hover:border-accent"
    
    # Test with opacity and modifiers
    assert str(bg_dui.neutral.opacity(50).hover) == "hover:bg-neutral/50"
    assert str(text_dui.base_content.hover.opacity(75)) == "hover:text-base-content/75"
    
    # Test responsive modifiers
    assert str(bg_dui.primary.md) == "md:bg-primary"
    assert str(text_dui.secondary.lg) == "lg:text-secondary"
    assert str(border_dui.base_300.sm) == "sm:border-base-300"
    
    # Test dark mode
    assert str(bg_dui.base_100.dark) == "dark:bg-base-100"
    assert str(text_dui.base_content.dark) == "dark:text-base-content"
    
    # Test chained modifiers
    assert str(bg_dui.primary.hover.dark) == "dark:hover:bg-primary"
    assert str(text_dui.error.md.hover.dark) == "dark:hover:md:text-error"
    
    # Test group and peer modifiers
    assert str(bg_dui.base_200.group("hover")) == "group-hover:bg-base-200"
    assert str(text_dui.success.peer("checked")) == "peer-checked:text-success"
    
    # Test arbitrary modifiers
    assert str(bg_dui.warning.aria("selected")) == "aria-selected:bg-warning"
    assert str(text_dui.info.data("active")) == "data-[active]:text-info"

# Run the test
test_colors_modifier_examples()

## Helper Functions

Utility functions for working with semantic colors:

In [ ]:
#| export
def get_all_semantic_colors() -> List[str]:
    """Get list of all daisyUI semantic color names."""
    return [c.value for c in SemanticColor]

In [ ]:
#| export
def get_brand_colors() -> List[str]:
    """Get list of brand semantic colors."""
    return [SemanticColor.PRIMARY.value, SemanticColor.SECONDARY.value, SemanticColor.ACCENT.value, SemanticColor.NEUTRAL.value]

In [ ]:
#| export
def get_base_colors() -> List[str]:
    """Get list of base semantic colors."""
    return [SemanticColor.BASE_100.value, SemanticColor.BASE_200.value, SemanticColor.BASE_300.value, SemanticColor.BASE_CONTENT.value]

In [ ]:
#| export
def get_status_colors() -> List[str]:
    """Get list of status semantic colors."""
    return [SemanticColor.INFO.value, SemanticColor.SUCCESS, SemanticColor.WARNING.value, SemanticColor.ERROR.value]

In [ ]:
# Test helper functions
colors = get_all_semantic_colors()
assert len(colors) == 20
assert "primary" in colors
assert "base-100" in colors
assert "error-content" in colors

brand = get_brand_colors()
assert len(brand) == 4
assert brand == ["primary", "secondary", "accent", "neutral"]

base = get_base_colors()
assert len(base) == 4
assert base == ["base-100", "base-200", "base-300", "base-content"]

status = get_status_colors()
assert len(status) == 4
assert status == ["info", "success", "warning", "error"]

print("✅ All semantic color helper functions work correctly!")

✅ All semantic color helper functions work correctly!


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()